In [1]:
from collections import defaultdict
from copy import deepcopy

In [2]:
test = '2333133121414131402'

In [3]:
def get_disk(data):
    disk = defaultdict(list)
    block = 0
    for idnt, d in enumerate(data):
        d = int(d)
        if idnt%2 == 1:
            ident = -1
        else:
            ident = idnt//2
        disk[ident].append([block,block+d])
        block += d
    return disk

def print_disk(disk):
    idents = list(disk.keys())
    idents.sort()
    disk_size = max([disk[idents[0]][-1][1], disk[idents[-1]][-1][1]])
    
    image = []
    for i in range(0, disk_size):
        image.append('.')
    
    for ident in idents:
        if ident == -1:
            continue
        for block in disk[ident]:
            for i in range(block[0], block[1]):
                image[i] = str(ident)
    print(''.join(image))
    
def join_blocks(blocks):
    if len(blocks) < 2:
        return blocks
    
    for i in range(len(blocks)-1, -1, -1):
        if blocks[i-1][1] == blocks[i][0]:
            blocks[i-1][1] = blocks[i][1]
            del blocks[i]
    return blocks

def get_disk_size(disk):
    maxi = []
    for key in disk.keys():
        disk[key].sort()
        maxi.append(disk[key][-1][1])
    return max(maxi)
    
def defrag_one_by_one(disk):
    disk_size = get_disk_size(disk)
    
    idents = list(disk.keys())
    idents.sort(reverse=True)
    
    prin = False
    
    for ident in idents:
        
        if ident == 6:
            prin = True
        
        if ident == -1:
            break
        
        if disk[ident][-1][1] < disk[-1][0][0]:
            break
        
        while True:
            free_space = disk[-1].pop(0)
            free_space_size = free_space[1]-free_space[0]

            ident_space = disk[ident].pop(-1)
            ident_space_size = ident_space[1]-ident_space[0]

            if ident_space_size < free_space_size:                
                disk[ident].append([free_space[0], free_space[0]+ident_space_size])
                disk[-1].append([free_space[0]+ident_space_size, free_space[1]])
                disk[-1].append(deepcopy(ident_space))
                
                disk[ident].sort()
                disk[ident] = join_blocks(disk[ident])
                disk[-1].sort()
                disk[-1] = join_blocks(disk[-1])
                
                break

            elif ident_space_size == free_space_size:               
                disk[ident].append(deepcopy(free_space))
                disk[-1].append(deepcopy(ident_space))
                
                disk[ident].sort()
                disk[ident] = join_blocks(disk[ident])
                disk[-1].sort()
                disk[-1] = join_blocks(disk[-1])
                
                break

            else: #if ident_space_size > free_space_size               
                disk[ident].append(deepcopy(free_space))
                disk[ident].append([ident_space[0], ident_space[1]-free_space_size])
                disk[-1].append([ident_space[1]-free_space_size, ident_space[1]])

            disk[ident].sort()
            disk[ident] = join_blocks(disk[ident])
            disk[-1].sort()
            disk[-1] = join_blocks(disk[-1])
                
            if len(disk[-1]) == 1 and disk[-1][-1][1] == disk_size:
                break

    return disk

def checksum(disk):
    czeksum = 0
    
    for ident in disk.keys():
        if ident == -1:
            continue
        for block in disk[ident]:
            for b in range(block[0], block[1]):
                czeksum += b*ident
    return czeksum

def defrag_files(disk):   
    idents = list(disk.keys())
    idents.sort(reverse=True)
    
    for ident in idents:
        ident_space = disk[ident][0]
        ident_space_size = disk[ident][0][1] - disk[ident][0][0]
        
        for i, free_space in enumerate(disk[-1]):
            if free_space[0] > ident_space[0]:
                break
            
            free_space_size = free_space[1]-free_space[0]
            
            if ident_space_size < free_space_size:                
                disk[ident][0] = [free_space[0], free_space[0]+ident_space_size]
                disk[-1][i] = [free_space[0]+ident_space_size, free_space[1]]
                disk[-1].append(deepcopy(ident_space))
                
                disk[ident].sort()
                disk[ident] = join_blocks(disk[ident])
                disk[-1].sort()
                disk[-1] = join_blocks(disk[-1])
                
                break

            elif ident_space_size == free_space_size:               
                disk[ident][0] = deepcopy(free_space)
                disk[-1][i] = deepcopy(ident_space)
                
                disk[ident].sort()
                disk[ident] = join_blocks(disk[ident])
                disk[-1].sort()
                disk[-1] = join_blocks(disk[-1])
                
                break
                
    return disk

def run(data, part=1, prin=False): 
    disk = get_disk(data)
    if prin:
        print_disk(disk)
    if part == 1:
        disk = defrag_one_by_one(disk)
    if part == 2:
        disk = defrag_files(disk)
        
    if prin:
        print_disk(disk)
    print('Part', part, 'result:', checksum(disk))

run(test, 1, True)
print()
run(test, 2, True)

00...111...2...333.44.5555.6666.777.888899
0099811188827773336446555566..............
Part 1 result: 1928

00...111...2...333.44.5555.6666.777.888899
00992111777.44.333....5555.6666.....8888..
Part 2 result: 2858


In [4]:
with open('input_day09.txt', 'r') as f:
    data = f.readlines()
    f.close()
    data = data[0].strip()

run(data, 1)
run(data, 2)

Part 1 result: 6291146824486
Part 2 result: 6307279963620
